# Analisi dei consumi elettrici di un utente domestico

* A. input data - 1. year 2.mese <br>
* B. data format- .csv
* C. results - 1. Consumo energetico totale [kWh] 2. Consumo energetico giornaliero per ogni giorno del mese [kWh]
> Consumi casa Silvia.

<div class="alert alert-block alert-info">
<b>Tip:</b> Use blue boxes (alert-info) for tips and notes. 
If it’s a note, you don’t have to include the word “Note”.
</div>

## 1. Preambolo

In [3]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Plots")

using CSV
using DataFrames
using Plots
using Dates

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


## 2. Importa dati consumi in un dataframe

read CSV 2021 <br

Esempio per importare 12 mesi da una cartella

In [28]:
#input
year = "2021"
months = ["gennaio","febbraio","marzo","aprile","maggio","giugno",
"luglio","agosto","settembre","ottobre","novembre","dicembre"]

#inizializza dataframe
df = []

# output - dataframe annuale
for i=1:12
    
    push!(df, CSV.read("data_"*year*"/ExportData_"*months[i]*".csv", DataFrame, delim=';',
    quoted=true, decimal=',', dateformat="dd/mm/yyyy",header=1))
    
end

# Osserva i risultati
# i warning si accendono perche la prima riga dei csv non termina con ;
# df è diventata una matrice con 12 righe
# esempio
# df[1] = data frame di gennaio
# df[3]= data frame di marzo
# df[2][:,95:end] = data frame di febbraio tutte le righe ultime tre colonne

# esempio di come usare il data frame del 2021
#df[2][:,95:end]

# esempio su come avere il vettore che contiene i nomi delle colonne
#names(df[2])

# esempio di come avere il vettore che contiene tutti gli elementi di una colonna
#df[2]."00:00-00:15"

# esempio codice per saper che giorno della settimana era es Monday
#Dates.dayname(df[1][10,1])

# Number of day in the mounth
#N_day = Int64(size(df[1],1))

## 3. Calcola il consumo annuale

In [29]:
# Consumo annuale totale [kWh]
consumi_annuali = sum(sum(df[i][:,j]) for j =2:97 for i=1:12)

783.3139999999999

## 4. Calcola i consumi mensili

In [30]:
# Consumo mensile totale [kWh]
consumi_mensili = zeros(12)
for i=1:12
 consumi_mensili[i] = sum(sum(df[i][:,j]) for j =2:97)
end
consumi_mensili

12-element Vector{Float64}:
 70.0
 56.33600000000002
 88.217
 62.38
 58.69599999999999
 52.347999999999985
 53.87799999999999
 43.21
 57.023
 72.74600000000004
 86.39700000000002
 82.08300000000001

## 5. Calcola il consumo giornaliero

In [31]:
#size(df[1],1)
#sum(df[1][18,2:97])
#df[1].Column98 = zeros(31)
#rename!(df[1], :Column98 => :ConumoGiornaliero)
df[1][:,95:end]

,23:15-23:30,23:30-23:45,23:45-00:00,Column98
,Float64,Float64,Float64,Missing
1,0.004,0.003,0.004,missing
2,0.004,0.003,0.004,missing
3,0.042,0.059,0.061,missing
4,0.071,0.074,0.067,missing
5,0.063,0.056,0.099,missing
6,0.007,0.007,0.007,missing
7,0.06,0.05,0.05,missing
8,0.032,0.024,0.02,missing
9,0.009,0.007,0.007,missing


In [25]:
for m = 1:12
    
    N_day = Int64(size(df[m],1))
    daily_E_cons = zeros(N_day)
    
    for g = 1:N_day
        daily_E_cons[g] = sum(df[m][g,2:97])
    end
    
    df[m].Column98 = daily_E_cons
    
end



df[5][5,98] 

1.73

In [27]:
sum(df[5].Column98)

58.695999999999984

In [106]:
df_daily = DataFrame()
df_daily.Giorno = df[1].Giorno
df_daily

,Giorno
,Date
1,2021-01-01
2,2021-01-02
3,2021-01-03
4,2021-01-04
5,2021-01-05
6,2021-01-06
7,2021-01-07
8,2021-01-08
9,2021-01-09


In [107]:
# Somma energia consumata giornaliera
df_daily = DataFrame()

for m = 1:12
    
    N_day = zeros(N_day)
    N_day = Int64(size(df[m],1))
    daily_E_cons = zeros(N_day)
    df_daily.Giorno = df[m].Giorno
    
    for d = 1:N_day
        daily_E_cons[d] = sum(df[m][d,2:97])
    end
    
    df_daily.Consumo_Giornaliero = daily_E_cons
end

df_daily

,Giorno,Consumo_Giornaliero
,Date,Float64
1,2021-12-01,3.857
2,2021-12-02,2.367
3,2021-12-03,0.742
4,2021-12-04,2.015
5,2021-12-05,5.69
6,2021-12-06,2.442
7,2021-12-07,3.286
8,2021-12-08,5.966
9,2021-12-09,2.957


In [ ]:
# crea feature giorno della settimana in dataframe 

for j = 1:N_day

DayOfTheWeek[j] = Dates.dayname(j,1)

days_of_the_week = DataFrame(DayOfTheWeek=[20, 40])

In [27]:
for j=1:size(df,2)
    df[j,98] = Dates.dayname(j,1)
end
df.23:00-23:15

LoadError: MethodError: no method matching dayname(::Int64, ::Int64)
[0mClosest candidates are:
[0m  dayname(::Integer; locale) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Dates/src/query.jl:150
[0m  dayname(::Integer, [91m::Dates.DateLocale[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Dates/src/query.jl:148